<a href="https://colab.research.google.com/github/adidam/rag-impl/blob/main/RGB-Ravi_Teja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## **Chunking the Dataset**

In [2]:
! pip install langchain
! pip install groq
! pip install -q langchain langchain-groq
! pip install transformers torch sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
# embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = "BAAI/LLM-Embedder"
# embedding_model = "BAAI/bge-large-en"
tokenizer = AutoTokenizer.from_pretrained(embedding_model)
embedder = SentenceTransformer(embedding_model)  # Pretrained sentence transformer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/28.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import langchain

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Initializing the context variable which later gets populated with retrieved chunks
context = ""

## **USER QUERY**

# **RAG System Evaluation on RGB Dataset**

In [5]:
# Clone the repository
!git clone https://github.com/chen700564/RGB.git
%cd RGB

# Additional LLM libraries (if required)
!pip install transformers sentence-transformers


Cloning into 'RGB'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 50 (delta 22), reused 34 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 8.16 MiB | 10.81 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/RGB


In [6]:
! pip install langchain
! pip install groq
! pip install -q langchain langchain-groq

In [7]:
from google.colab import userdata
api_key = userdata.get('my_key')


In [8]:
import json
import langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0.3, groq_api_key=api_key, model_name="llama3-8b-8192")

prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below, strictly adhering to the
      information presented in the following documents.
      Do not generate any text beyond what is explicitly stated in the documents.

      Context: {context}

      Question: {query}

      Answer:
      """
    )
chain = prompt | chat

In [9]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# **RAG prompt to LLM as given in the research paper 239_01431** #

In [22]:
system_prompt = """
  You are an accurate and reliable AI assistant that can
answer questions with the help of external documents.
Please note that external documents may contain noisy
or factually incorrect information. If the information in
the document contains the correct answer, you will give
an accurate answer. If the information in the document
does not contain the answer, you will generate ’I can not
answer the question because of the insufficient
information in documents.‘ If there are inconsistencies
with the facts in some of the documents, please generate
the response 'There are factual errors in the provided
documents.' and provide the correct answer.
"""
user_prompt = """
Document:\n{context} \n\nQuestion:\n{query}
"""



In [23]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_groq import ChatGroq
import json

system_message = SystemMessagePromptTemplate.from_template(system_prompt)
user_message = HumanMessagePromptTemplate.from_template(user_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

In [24]:
# chat = ChatGroq(temperature=0.3, groq_api_key=api_key, model_name="llama3-8b-8192")
chat = ChatGroq(temperature=0.3, groq_api_key=api_key, model_name="deepseek-r1-distill-llama-70b")
llm_chain = chat_prompt | chat

In [25]:
# Noise Robustness Evaluation

input_file = "/content/RGB/data/en_refine.json"
output_file = "/content/RGB/data/en_refine_fixed.json"

with open(input_file, "r") as f:
    lines = f.readlines()

# Wrap each line in an array
json_objects = [line.strip() for line in lines if line.strip()]
wrapped_json = f"[{','.join(json_objects)}]"

# Write to a new file
with open(output_file, "w") as f:
    f.write(wrapped_json)

# Load the fixed JSON file
with open(output_file, "r") as f:
    data = json.load(f)

for query in data[:5]:
    context = ""
    pos_docs = ""
    neg_docs = ""
    noisy_query = query["query"]
    expected_answer = query["answer"]

    pos_docs = " ".join(query["positive"]) if isinstance(query["positive"], list) else query["positive"]
    neg_docs = " ".join(query["negative"]) if isinstance(query["negative"], list) else query["negative"]

    pos = sent_tokenize(pos_docs)
    neg = sent_tokenize(neg_docs)

    context = "\n".join(pos[:3])

    context = context.join(neg[:2])

    print("context>>>>>",context)
    groq_response_noise = llm_chain.invoke({"context": context, "query": noisy_query})

    print(f"Query: {noisy_query}\nResponse: {groq_response_noise.content}\nExpected: {expected_answer}\n")


context>>>>> Feb 10, 2022                                 Runtime:                                   2h 0m                                Carole King                                                                                          Self                                                             James Taylor                                                                                          Self                                                             Danny Kortchmar                                                                                          Self                                                             Russ Kunkel                                                                                          Self                                                             Lee Sklar                                                                                          Self                                                             Frank Marshall               

In [ ]:
#Noise robust evluation wiht accuracy

import json
import random
from nltk.tokenize import sent_tokenize
from itertools import chain as itertools_chain
import time
import random

input_file = "/content/RGB/data/en_refine.json"
output_file = "/content/RGB/data/en_refine_fixed.json"

# Fix JSON formatting
with open(input_file, "r") as f:
    lines = f.readlines()

json_objects = [line.strip() for line in lines if line.strip()]
wrapped_json = f"[{','.join(json_objects)}]"

with open(output_file, "w") as f:
    f.write(wrapped_json)

# Load the fixed JSON file
with open(output_file, "r") as f:
    data = json.load(f)

def flatten_list(nested_list):
    """Flatten a nested list into a single list of strings."""
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        else:
            flattened.append(item)
    return flattened

def evaluate_accuracy(data, noise_ratio):
    correct = 0
    #total = len(data)
    total = 20

    for query in data[:total]:
        context = ""
        noisy_query = query["query"]
        expected_answer = query["answer"]

        # Ensure expected_answer is a properly formatted string
        expected_answer = flatten_list(expected_answer)

        # Get all positive and negative documents
        pos_docs = " ".join(flatten_list(query["positive"]))
        neg_docs = " ".join(flatten_list(query["negative"]))

        # Tokenize sentences
        pos = sent_tokenize(pos_docs)
        neg = sent_tokenize(neg_docs)

        # Total documents to consider (limit to 10)
        total_docs = 10

        # Calculate the number of positive and negative sentences to be considered
        num_pos_sentences = round(total_docs * (1 - noise_ratio))  # Positive sentences
        num_neg_sentences = total_docs - num_pos_sentences  # Negative sentences

        print("num_pos_sentences>>>", num_pos_sentences)
        print("num_neg_sentences>>>", num_neg_sentences)

        # Select the required number of positive and negative sentences
        selected_pos = pos[:num_pos_sentences]
        selected_neg = neg[:num_neg_sentences]

        # If not enough positive or negative docs, fill in from available documents
        if len(selected_pos) < num_pos_sentences:
            selected_pos.extend(random.choices(pos, k=num_pos_sentences - len(selected_pos)))
        if len(selected_neg) < num_neg_sentences:
            selected_neg.extend(random.choices(neg, k=num_neg_sentences - len(selected_neg)))

        # Introduce noise to negative sentences (if noise ratio > 0)
        if noise_ratio > 0:
            num_noisy_sentences = round(len(selected_neg) * noise_ratio)
            noise_candidates = ["Random noise sentence", "Irrelevant information", "Unrelated text"]
            selected_neg[:num_noisy_sentences] = random.choices(noise_candidates, k=num_noisy_sentences)

        # Combine selected positive and negative sentences for context
        context = "\n".join(selected_pos + selected_neg)

        # Get response from the model (LLM chain invocation)
        groq_response_noise = llm_chain.invoke({"context": context, "query": noisy_query})
        model_answer = groq_response_noise.content.strip().lower()

        # Debugging outputs for expected and model answer
        print(f"Query: {noisy_query}\nResponse: {model_answer}\nExpected: {expected_answer}\n")

        # Check if the expected answer is a substring of the model's response
        for item in expected_answer:
          if item.lower() in model_answer.lower():
            correct += 1
            break

        # Add delay to avoid exceeding rate limit
        time.sleep(5)  # Adjust delay based on API rate limits

    # Calculate accuracy
    print("correct>>>",correct)
    print("total>>>",total)
    accuracy = (correct / total) * 100
    return accuracy



# Evaluate under different noise ratios
#noise_levels = [0.0, 0.2, 0.4, 0.6, 0.8]
noise_levels = [0.0, 0.2, 0.8]
results = {}

# Ensure you pass the correct LLM pipeline as `chain`
for noise in noise_levels:
    accuracy = evaluate_accuracy(data, noise)
    results[noise] = accuracy
    print(f"Noise Ratio: {noise}, Accuracy: {accuracy:.2f}%")




num_pos_sentences>>> 10
num_neg_sentences>>> 0
Query: When is the premiere of 'Carole King & James Taylor: Just Call Out My Name'?
Response: <think>
okay, i need to figure out when the premiere of 'carole king & james taylor: just call out my name' was. let me look through the document provided. 

first, i see multiple mentions of the premiere date. it says the film premiered on january 2, 2022, at 9:00pm et/pt on cnn. this information is repeated a few times in the document, so it seems consistent. 

i don't see any conflicting dates or times mentioned. all the sections about the premiere line up with january 2, 2022. there's also information about availability on demand starting from january 3, but the premiere itself is clearly stated as january 2. 

i don't notice any factual errors in the provided documents. all the details about the premiere date, time, and network are the same each time they're mentioned. 

so, putting it all together, the premiere was on january 2, 2022, at 9:0

In [18]:
# Negative Rejection Evaluation

for query in data[5:10]:
    context = ""
    irrelevant_query = query["query"]
    expected_answer = query["answer"]

    neg_docs = "\n".join(query["negative"]) if isinstance(query["negative"], list) else query["negative"]

    neg = sent_tokenize(neg_docs)

    context = "\n".join(pos[:5])

    print("context>>>>>",context)
    groq_response_negetive = chain.invoke({"context": context, "query": irrelevant_query})

    print(f"Query: {irrelevant_query}\nResponse: {groq_response_negetive.content}\nExpected: {expected_answer}\n")


context>>>>> The EV maker doesn't disclose U.S.-only sales figures but claims sales for the year were up an amazing 87 percent in a turbulent marketplace.
Anyone who thinks the legacy automakers are catching Tesla in the electric-vehicle sales race should take a quick look at some recently released figures.
Over the weekend, Tesla announced it delivered 308,600 vehicles in the fourth quarter of 2021.
Model S and Model X EVs made up 11,750 of those deliveries, while the bulk—296,850—were Model 3 and Model Y deliveries.
These numbers brought Tesla's 2021 full-year deliveries to 936,172 vehicles.
Query: Which company acquired ShowBiz Cinemas?
Response: There is no mention of ShowBiz Cinemas or any acquisition in the provided documents. The documents only discuss Tesla's electric vehicle sales figures.
Expected: ['EVO Entertainment Group']

context>>>>> The EV maker doesn't disclose U.S.-only sales figures but claims sales for the year were up an amazing 87 percent in a turbulent marketpla

In [ ]:
# Information Integration Evaluation
from sentence_transformers import SentenceTransformer, util

input_file = "/content/RGB/data/en_int.json"
output_file = "/content/RGB/data/en_int_fixed.json"

with open(input_file, "r") as f:
    lines = f.readlines()

# Wrap each line in an array
json_objects = [line.strip() for line in lines if line.strip()]
wrapped_json = f"[{','.join(json_objects)}]"

# Write to a new file
with open(output_file, "w") as f:
    f.write(wrapped_json)

# Load the fixed JSON file
with open(output_file, "r") as f:
    data = json.load(f)

retriever = SentenceTransformer("all-MiniLM-L6-v2")

for query in data[:5]:
    integrated_query = query["query"]
    expected_answer = query["answer"]

    pos_docs = [doc for doc_list in query["positive"] for doc in doc_list]

    # The positive docs already have relevant data why sort again and pick docs?
    # docs_embeddings = retriever.encode(pos_docs, convert_to_tensor=True)
    # query_embedding = retriever.encode(integrated_query, convert_to_tensor=True)
    # scores = util.pytorch_cos_sim(query_embedding, docs_embeddings)
    # scores = scores.flatten()
    # print(f">>>>>> Scores: {scores}")
    # top_docs = [pos_docs[i] for i in scores.argsort(descending=True)]

    context = "\n".join(pos_docs)
    # response = model(context, max_length=50)[0]["generated_text"]
    print("context>>>>>",context)
    groq_response_integrated = chain.invoke({"context": context, "query": integrated_query})

    print(f"Query: {integrated_query}\nResponse: {groq_response_integrated.content}\nExpected: {expected_answer}\n")


In [ ]:
# Counter Factual Robusiness evaluation

input_file = "/content/RGB/data/en_fact.json"
output_file = "/content/RGB/data/en_fact_fixed.json"

with open(input_file, "r") as f:
    lines = f.readlines()

# Wrap each line in an array
json_objects = [line.strip() for line in lines if line.strip()]
wrapped_json = f"[{','.join(json_objects)}]"

# Write to a new file
with open(output_file, "w") as f:
    f.write(wrapped_json)

# Load the fixed JSON file
with open(output_file, "r") as f:
    data = json.load(f)


for query in data[:5]:  # Limiting to first 5 queries for testing
    counterfactual_query = query["query"]
    expected_behavior = query["answer"]
    fake_answer = query["fakeanswer"]
    context = query["positive_wrong"] + query["positive"] + query["negative"]

    response = chain.invoke({"context": context, "query": counterfactual_query})
    #response = model(counterfactual_query, max_length=50)[0]["generated_text"]
    print(f"Query: {counterfactual_query}\nResponse: {response}\nExpected: {expected_behavior}\nFake Answer: {fake_answer}\n")


Query: Super Bowl 2021 location
Response: content='Based on the provided document, the answer to the question is:\n\nRaymond James Stadium in Glendale, Arizona.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 275, 'total_tokens': 299, 'completion_time': 0.02, 'prompt_time': 0.046140257, 'queue_time': 0.019813660000000004, 'total_time': 0.066140257}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None} id='run-af72c747-53dc-4093-8be6-dcdb6f4822eb-0' usage_metadata={'input_tokens': 275, 'output_tokens': 24, 'total_tokens': 299}
Expected: Tampa, Florida
Fake Answer: Glendale, Arizona

Query: Which country won the most medals at the 2018 Winter Olympics?
Response: content='According to the provided document, the United States won the most medals at the 2018 Winter Olympics, with a total of 39 medals.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 30